# Section 6.2: More Detailed Comparison of AAE and PPI-based Methods
 **This notebook runs the experiments in Section 6.2, which provides a more detailed comparison of AAE and PPI-based methods.**

### Import necessary packages

In [39]:
import os, sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))
import numpy as np
import pandas as pd
import pickle as pkl
from ppi_py.datasets import load_dataset
from ppi_py import ppi_logistic_pointestimate, logistic
from ppi_utils import *
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from tqdm import tqdm
from scipy.optimize import brentq
from aae import *

### Define utility functions
Define the one-sample t-test function

In [33]:
def compute_bias_reduction(df, true_theta):
    # Group by method and m, compute MAPE for each group
    df_grouped = df.groupby(['method', 'm']).agg({
        'point_estimate': lambda x: np.mean(np.mean([
            np.abs((est - true_theta)/(true_theta)) * 100
            for est in x.values
        ], axis=0))
    }).reset_index()

    # Rename column for clarity
    df_grouped = df_grouped.rename(columns={'point_estimate': 'mape'})  

    # Compute the difference between the methods and the human-data-only
    diff_data = []
    for m in ms:
        mape_human = df_grouped[(df_grouped['method'] == 'Human-data-only') & (df_grouped['m'] == m)]['mape'].values[0]
        for method in ['AAE', 'PPI', 'PPI++']:
            mape_method = df_grouped[(df_grouped['method'] == method) & (df_grouped['m'] == m)]['mape'].values[0]
            diff = mape_method - mape_human
            diff_data.append({'method': method, 'm': m, 'mape_diff': diff})
            
    df_diff = pd.DataFrame(diff_data)
    # Pivot to get the desired table: rows are methods, columns are m
    df_diff_table = df_diff.pivot(index='m', columns='method', values='mape_diff')
     # Display df_grouped with 2 decimal places
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    display(df_diff_table)
    return None


In [34]:
from scipy import stats

def perform_t_test(df, method1, method2, m, true_theta):
    """
    Perform t-test comparing MAPE differences between two methods at given m.
    
    Args:
        df: DataFrame containing results
        method1: First method to compare (str)
        method2: Second method to compare (str) 
        m: Primary set size to analyze (int)
        true_theta: True parameter value
        
    Returns:
        t_stat: t-statistic from t-test
        p_value: p-value from t-test
        mean_diff: Mean MAPE difference between methods
    """
    # Get results for specified m
    n_results = df[df['m'] == m]
    method1_estimates = n_results[n_results['method'] == method1]['point_estimate'].values
    method2_estimates = n_results[n_results['method'] == method2]['point_estimate'].values
    
    # Calculate MAPE differences (method1 - method2)
    mape_diffs = np.array([
        np.mean(np.abs((m1 - true_theta)/true_theta)) - 
        np.mean(np.abs((m2 - true_theta)/true_theta))
        for m1, m2 in zip(method1_estimates, method2_estimates)
    ])

    # Perform one-sample t-test
    t_stat, p_value = stats.ttest_1samp(mape_diffs, 0, alternative='less')
    
    return t_stat, p_value, np.mean(mape_diffs)


## Section 6.2.1: Private health insurance dataset

The goal is to investigate the quantitative effect of income on the procurement of private health insurance using US census data. The target of inference is the logistic regression coefficient when regressing the binary indicator of health insurance on income. The data from California in the year 2019 is downloaded through the Folktables interface (1). Predictions of health insurance are made by training a gradient boosting tree via XGBoost (2) on the previous year’s data.

1. F. Ding, M. Hardt, J. Miller, L. Schmidt, “Retiring adult: New datasets for fair machine learning” in Advances in Neural Information Processing Systems 34 (2021), pp. 6478–6490.
2. T. Chen, C. Guestrin, “XGBoost: A scalable tree boosting system” in Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining (2016), pp. 785–794.

### Import the census healthcare data set

Load the data. The data set contains reported indicators of health insurance (```Y```), predicted indicators of health insurance (```Yhat```), and reported income (```X```).

In [4]:
dataset_folder = "./data/census/"
data = load_dataset(dataset_folder, "census_healthcare")
Y_total = data["Y"]
Yhat_total = data["Yhat"]
X_total = data["X"]

# Normalization of X
X_total[:,0] = (X_total[:,0] - X_total[:,0].min()) / (X_total[:,0].max() - X_total[:,0].min())
X_total[:,1] = X_total[:,1] / X_total[:,1].max()

### Experiment setup

Specify the range of values for the primary set size (```ms```), the auxiliary set size (```n```) and number of trials (```num_trials```).

Compute the ground-truth value of the estimand.

In [ ]:
n_total = Y_total.shape[0]  # Total number of labeled examples
ms = np.array([100, 250, 500, 750, 1000]).astype(int)  # Test for different primary set sizes
n = 2000    # size of the auxiliary set
num_trials = 50
optimizer_options = {
    "ftol": 1e-5,
    "gtol": 1e-5,
    "maxls": 10000,
    "maxiter": 10000,
}

# Saving results settings
# WARNING::: If setting save_results to TRUE, the previous results will be OVERWRITTEN.
save_results = False # TRUE to save results to pickle file
# File name to save results: '{folder}/results_census_{n_total}_{num_trials}.pkl'
resfile = 'res/res_census_3000_50.pkl' 

# Compute ground truth
true_theta = (
    LogisticRegression(
        penalty=None,
        solver="lbfgs",
        max_iter=10000,
        tol=1e-15,
        fit_intercept=False,
    )
    .fit(X_total, Y_total)
    .coef_.squeeze()
)

### Running Experiments

In [ ]:
# Run prediction-powered inference and human-data-only inference for many values of m
results = []
for i in range(ms.shape[0]):
    for j in tqdm(range(num_trials)):
        # Prediction-Powered Inference
        m = ms[i]
        rng = np.random.RandomState(j)
        rand_idx = rng.permutation(n_total)
        _X, _X_unlabeled = X_total[rand_idx[:m]], X_total[rand_idx[m:m+n]]
        _Y = Y_total[rand_idx[:m]]
        _Yhat, _Yhat_unlabeled = Yhat_total[rand_idx[:m]], Yhat_total[rand_idx[m:m+n]]
    
        # PPI point estimate
        ppi_pe = ppi_logistic_pointestimate(
            _X,
            _Y, 
            _Yhat,
            _X_unlabeled,
            _Yhat_unlabeled,
            lam=1,
            optimizer_options=optimizer_options
        )

        # PPI++ point estimate
        ppi_opt_pe = ppi_logistic_pointestimate(
            _X,
            _Y, 
            _Yhat,
            _X_unlabeled,
            _Yhat_unlabeled,
            optimizer_options=optimizer_options
        )
        
        # Human-data-only point estimate
        human_data_only_pe = logistic(_X, _Y)

        # Append results
        results += [
            pd.DataFrame(
                [
                    {
                        "method": "Human-data-only",
                        "m": m,
                        "trial": j,
                        "point_estimate": human_data_only_pe,
                    }
                ]
            )
        ]

        results += [
            pd.DataFrame(
                [
                    {
                        "method": "PPI",
                        "m": m,
                        "trial": j,
                        "point_estimate": ppi_pe,
                    }
                ]
            )
        ]

        results += [
            pd.DataFrame(
                [
                    {
                        "method": "PPI++",
                        "m": m,
                        "trial": j,
                        "point_estimate": ppi_opt_pe,
                    }
                ]
            )
        ]

In [ ]:
# Run AAE for many values of m
X_total1 = [np.array([np.concatenate(([1], x)), np.array([1,0,0])]) for x in X_total]
X_total1_flat = flatten_full(X_total1)
Yhat_total1 = (Yhat_total > 0.5).astype(int)

for i in range(ms.shape[0]):
    for j in tqdm(range(num_trials)):
        m = ms[i]
        rng = np.random.RandomState(j)
        rand_idx = rng.permutation(n_total)
        _X1, _X_unlabeled1 = np.array(X_total1)[rand_idx[:m]], np.array(X_total1)[rand_idx[m:m+n]]
        _X_unlabeled1_flat = np.array(X_total1_flat)[rand_idx[m:m+n]]
        _Y = Y_total[rand_idx[:m]]
        _Yhat1, _Yhat_unlabeled1 = Yhat_total1[rand_idx[:m]], Yhat_total1[rand_idx[m:m+n]]

        # AAE point estimate
        g_model = LogisticRegression(penalty=None, solver='lbfgs', max_iter=10000, tol=1e-15, fit_intercept=False)
        aae_pe = - aae(_X1, _Y, _Yhat1, _X_unlabeled1, _Yhat_unlabeled1, _X_unlabeled1_flat, g_model, concat=1, n_epochs=5000, lr=1e-2) 
        results += [
            pd.DataFrame(
                [
                    {
                        "method": "AAE",
                        "m": m,
                        "trial": j,
                        "point_estimate": aae_pe,
                    }
                ]
            )
        ]

# Save results to pickle file
if save_results:
    with open(resfile, 'wb') as f:  
        pkl.dump(results, f)

### Analyzing Results

#### Load Experiment Results

In [14]:
# Load results from pickle file
with open(resfile, 'rb') as f:
    results = pkl.load(f)

#### Compute the Bias Reduction from the Humand-data-only Estimators

In [15]:
df = pd.concat(results, axis=0, ignore_index=True)

compute_bias_reduction(df, true_theta)

method,AAE,PPI,PPI++
m,,,
100,-65.86,-22.29,-15.22
250,-35.98,-12.70,-4.70
500,-29.93,-7.58,-2.42
750,-10.74,-2.13,-1.05
1000,-11.87,-4.57,-1.15


#### Perform One-Sample t-Test

In [16]:
results_list = []
for m in ms:
    # AAE vs PPI++
    t_stat, p_val, diff = perform_t_test(df, 'AAE', 'PPI++', m, true_theta)
    results_list.append({
        'comparison': 'AAE v.s. PPI++',
        'm': m,
        't_statistic': t_stat,
        'p_value': p_val,
        'difference': diff
    })
    
    # AAE vs PPI
    t_stat, p_val, diff = perform_t_test(df, 'AAE', 'PPI', m, true_theta)
    results_list.append({
        'comparison': 'AAE v.s. PPI',
        'm': m,
        't_statistic': t_stat,
        'p_value': p_val, 
        'difference': diff
    })
    
    # AAE vs Human-data-only
    t_stat, p_val, diff = perform_t_test(df, 'AAE', 'Human-data-only', m, true_theta)
    results_list.append({
        'comparison': 'AAE v.s. Human-data-only',
        'm': m,
        't_statistic': t_stat,
        'p_value': p_val,
        'difference': diff
    })

# Convert to dataframe
t_test_results = pd.DataFrame(results_list)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
t_test_results



,comparison,m,t_statistic,p_value,difference
0,AAE v.s. PPI++,100,-2.58,0.01,-0.51
1,AAE v.s. PPI,100,-1.80,0.04,-0.44
2,AAE v.s. Human-data-only,100,-3.36,0.00,-0.66
3,AAE v.s. PPI++,250,-2.98,0.00,-0.31
4,AAE v.s. PPI,250,-1.99,0.03,-0.23
5,AAE v.s. Human-data-only,250,-3.68,0.00,-0.36
6,AAE v.s. PPI++,500,-2.71,0.00,-0.28
7,AAE v.s. PPI,500,-2.41,0.01,-0.22
8,AAE v.s. Human-data-only,500,-3.26,0.00,-0.30
9,AAE v.s. PPI++,750,-1.26,0.11,-0.10


## Section 6.2.2: Synthetic Data Experiment

The goal is to investigate the quantitative effect of the number of covariates on the performance of PPI-based methods and AAE.

### Import the simulated data set

Load the data. The data set contains true choice label (```y```), augmented choice label (```y_aug```), and feature vectors (```X```).

In [73]:
dx = 8 # number of covariates, can be any value in [2,4,6,8,10]
n_total = 1200
with open(f'./data/sim/train_sim_{dx}_{n_total}.pkl', 'rb') as f:
    data = pkl.load(f)[0]
Y_total = data["y"]
Yhat_total = data["y_aug"]
X_total1 = data["X"]
true_theta = data["true_theta"]
X_total = [X_total1[i][1, 1:] - X_total1[i][0, 1:] for i in range(len(X_total1))]
X_total = np.array(X_total)

### Experimental setup

1. Specify the range of values for the labeled data set size (```ms```), and number of trials (```num_trials```).
2. Setup optimizer options for PPI
3. Setup first-stage estimator for AAE

In [ ]:
n_total = Y_total.shape[0]  # Total number of labeled examples
ms = np.array([100]).astype(int)  # Primary set size
n = 1000
num_trials = 50
optimizer_options = {
    "ftol": 1e-5,
    "gtol": 1e-5,
    "maxls": 10000,
    "maxiter": 10000,
}
# Saving results settings
# WARNING::: If setting save_results to TRUE, the previous results will be OVERWRITTEN.
save_results = False # TRUE to save results to a pickle file

### Running the Experiments

In [75]:
# Run prediction-powered inference and human-data-only inference
results = []
for i in range(ms.shape[0]):
    for j in tqdm(range(num_trials)):
        m = ms[i]
        rng = np.random.RandomState(j)
        rand_idx = rng.permutation(n_total)
        _X, _X_unlabeled = X_total[rand_idx[:m]], X_total[rand_idx[m:m+n]]
        _Y = Y_total[rand_idx[:m]]
        _Yhat, _Yhat_unlabeled = Yhat_total[rand_idx[:m]],Yhat_total[rand_idx[m:m+n]]
    
        # PPI point estimate
        ppi_pe = ppi_logistic_pointestimate(
            _X,
            _Y, 
            _Yhat,
            _X_unlabeled,
            _Yhat_unlabeled,
            lam=1,
            optimizer_options=optimizer_options
        )

        # PPI++ point estimate
        ppi_opt_pe = ppi_logistic_pointestimate(
            _X,
            _Y, 
            _Yhat,
            _X_unlabeled,
            _Yhat_unlabeled,
            optimizer_options=optimizer_options
        )
        
        # Human-data-only point estimate
        human_data_only_pe = logistic(_X, _Y)

        # Append results
        results += [
            pd.DataFrame(
                [
                    {
                        "method": "Human-data-only",
                        "m": m,
                        "trial": j,
                        "point_estimate": human_data_only_pe,
                    }
                ]
            )
        ]

        results += [
            pd.DataFrame(
                [
                    {
                        "method": "PPI",
                        "m": m,
                        "trial": j,
                        "point_estimate": ppi_pe,
                    }
                ]
            )
        ]

        results += [
            pd.DataFrame(
                [
                    {
                        "method": "PPI++",
                        "m": m,
                        "trial": j,
                        "point_estimate": ppi_opt_pe,
                    }
                ]
            )
        ]

100%|██████████| 50/50 [00:00<00:00, 162.14it/s]


In [76]:
# Run AAE
X_total1_flat = flatten_full(X_total1)
Yhat_total1 = (Yhat_total > 0.5).astype(int)

for i in range(ms.shape[0]):
    for j in tqdm(range(num_trials)):
        m = ms[i]
        rng = np.random.RandomState(j)
        rand_idx = rng.permutation(n_total)
        _X1, _X_unlabeled1 = np.array(X_total1)[rand_idx[:m]], np.array(X_total1)[rand_idx[m:m+n]]
        _X_unlabeled1_flat = np.array(X_total1_flat)[rand_idx[m:m+n]]
        _Y = Y_total[rand_idx[:m]]
        _Yhat1, _Yhat_unlabeled1 = Yhat_total1[rand_idx[:m]], Yhat_total1[rand_idx[m:m+n]]

        # AAE point estimate
        g_model = GradientBoostingClassifier(n_estimators=5, learning_rate=0.1, max_depth=2, random_state=0)
        aae_pe = aae(_X1, _Y, _Yhat1, _X_unlabeled1, _Yhat_unlabeled1, _X_unlabeled1_flat, g_model, concat=1, n_epochs=1000, lr=1e-2)
        
        results += [
            pd.DataFrame(
                [
                    {
                        "method": "AAE",
                        "m": m,
                        "trial": j,
                        "point_estimate": aae_pe,
                    }
                ]
            )
        ]

# Save results to pickle file
if save_results:    
    with open(f'res/res_sim_{dx}_{n}_{num_trials}.pkl', 'wb') as f:
        pkl.dump(results, f)

100%|██████████| 50/50 [00:17<00:00,  2.91it/s]


### Analyzing Results

#### Load saved experimental results

In [77]:
# Load previous saved results and true theta
dx = 8 # Specify the dimension d, can be any value in [2,4,6,8,10]
n = 1000
n_total = 1200
num_trials = 50

with open(f'res/res_sim_{dx}_{n}_{num_trials}.pkl', 'rb') as f:
    results = pkl.load(f)

with open(f'./data/sim/train_sim_{dx}_{n_total}.pkl', 'rb') as f:
    true_theta = pkl.load(f)[0]['true_theta']

#### Compute Bias Reductions from the Human-data-only estimator

In [78]:
df = pd.concat(results, axis=0, ignore_index=True)

compute_bias_reduction(df, true_theta)

method,AAE,PPI,PPI++
m,,,
100,-145.71,-5.95,-66.36


#### Perform One-Sample t-Test

In [79]:
# Store t-test results in a dataframe
results_list = []
for m in ms:
    # AAE vs PPI++
    t_stat, p_val, diff = perform_t_test(df, 'AAE', 'PPI++', m, true_theta)
    results_list.append({
        'comparison': 'AAE v.s. PPI++',
        'm': m,
        't_statistic': t_stat,
        'p_value': p_val,
        'difference': diff
    })
    
    # AAE vs PPI
    t_stat, p_val, diff = perform_t_test(df, 'AAE', 'PPI', m, true_theta)
    results_list.append({
        'comparison': 'AAE v.s. PPI',
        'm': m,
        't_statistic': t_stat,
        'p_value': p_val, 
        'difference': diff
    })
    
    # AAE vs Human-data-only
    t_stat, p_val, diff = perform_t_test(df, 'AAE', 'Human-data-only', m, true_theta)
    results_list.append({
        'comparison': 'AAE v.s. Human-data-only',
        'm': m,
        't_statistic': t_stat,
        'p_value': p_val,
        'difference': diff
    })

# Convert to dataframe
t_test_results = pd.DataFrame(results_list)
pd.set_option('display.float_format', lambda x: '%.0e' % x if abs(x) < 1e-3 else '%.3f' % x)
t_test_results[['comparison', 't_statistic', 'p_value', 'difference']]

,comparison,t_statistic,p_value,difference
0,AAE v.s. PPI++,-6.540,2e-08,-0.793
1,AAE v.s. PPI,-8.061,8e-11,-1.398
2,AAE v.s. Human-data-only,-7.661,3e-10,-1.457
